根據把首頁往下滑，發現網站會對server送一個GET請求下一個30篇文章
- 2018(G) https://ithelp.ithome.com.tw/articles/10203216
- 2019(G) https://tlyu0419.github.io/2019/04/06/Crawl-Dcard/
- 2020 https://blog.jiatool.com/posts/dcard_api_v2/

### API root

- 看板資訊 http://dcard.tw/_api/forums
- 文章資訊 http://dcard.tw/_api/forums/{看板名稱}/posts
- 文章內文	http://dcard.tw/_api/posts/{文章編號}
- 文章內引用連結	http://dcard.tw/_api/posts/{文章編號}/links
- 文章內留言	http://dcard.tw/_api/posts/{文章編號}/comments

### API wirte
- 文章數量 https://www.dcard.tw/_api/posts?limit=100
- 文章分頁 http://dcard.tw/_api/posts/?before=文章id (before 回傳由新到舊 after舊到新)
- 文章留言 http://dcard.tw/_api/posts/文章id/comments
- 留言分頁 http://dcard.tw/_api/posts/文章id/comments?after=floor數字
- 熱門文章 https://dcard.tw/_api/posts?popular=true

### others
文章資訊(meta) 與 文章內留言 預設使用熱門度 (popularity) 作為排序，而且一次請求 (request) 中只回應 30 筆。
* 想要讓這兩項使用 時間 排序，可在 GET 參數中加入 popular=false
* 欲取得更多的 文章資訊(meta)，可以使用 before={某文章編號}來獲得早於 #{某文章編號} 的另外 30 筆 資訊。
* 欲取得更多的 文章內留言，可以使用 after={某樓層} 來獲得大於 #{某樓層} 的另外 30 筆 留言。

In [1]:
import requests
import json
import pandas as pd    # 引用套件並縮寫為 pd
from bs4 import BeautifulSoup
# https://www.dcard.tw/f/makeup

GET 美妝版首頁，Use BeautifulSoup with html, wrtie a txt

 

In [2]:
p = requests.Session()                             # 把這一次的request都算在同一個session裡
url=requests.get("https://www.dcard.tw/f/makeup")  # 請求的網址
soup = BeautifulSoup(url.text,"html.parser")
sel = soup.select("h2.tgn9uw-2 a")
sel

#for link in sel:
#    a.append(link["href"])
    #print('https://www.dcard.tw'+link.get('href'))

[<a class="tgn9uw-3 iuyCWN" href="/f/makeup/p/235164744"><span>#分享 2020空空賞（混油/酒糟/敏感肌/油頭愛用品）</span></a>,
 <a class="tgn9uw-3 iuyCWN" href="/f/makeup/p/235163281"><span>#分享 2021新年14樣空凹大賞✨</span></a>,
 <a class="tgn9uw-3 iuyCWN" href="/f/makeup/p/235163815"><span>#你不能錯過 celefit 9宮格的三種眼妝👀✨</span></a>,
 <a class="tgn9uw-3 iuyCWN" href="/f/makeup/p/235166281"><span>必須火的色號🔥M.A.C 絲柔粉霧唇釉 #977</span></a>,
 <a class="tgn9uw-3 iuyCWN" href="/f/makeup/p/235162079"><span>黑 某婚紗店化妝師</span></a>,
 <a class="tgn9uw-3 iuyCWN" href="/f/makeup/p/235163109"><span>#分享 💄26歲女孩彩妝強迫症收納</span></a>,
 <a class="tgn9uw-3 iuyCWN" href="/f/makeup/p/235162817"><span>#分享 小凱老師眼妝學院 課程心得</span></a>,
 <a class="tgn9uw-3 iuyCWN" href="/f/makeup/p/235162715"><span>#買買買 口罩期卯起來養水煮蛋肌</span></a>,
 <a class="tgn9uw-3 iuyCWN" href="/f/makeup/p/235162702"><span>#試色 #分享 heme必收腮紅色號😍</span></a>,
 <a class="tgn9uw-3 iuyCWN" href="/f/makeup/p/235163884"><span>U#屈臣氏 清貨分享~</span></a>,
 <a class="tgn9uw-3 iuyCWN" href="/f/makeup/p/235166378"

In [3]:
# slope the title
def Catch(num):
    try:
        url = 'https://www.dcard.tw/_api/posts/'+str(num)  #api post with ID
        resq = requests.get(url)
        rejs = resq.json()
        return(pd.DataFrame(
            data=
            [{'id':rejs['id'],
              'title':rejs['title'],
              'content':rejs['content'],
              'excerpt':rejs['excerpt'],
              'createdAt':rejs['createdAt'],
              'updatedAt': rejs['updatedAt' ],
              'forumName':rejs['forumName'],
              'forumAlias':rejs['forumAlias'],
              'gender':rejs['gender'],
              'topics':rejs['topics']}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )
    except:
        print('data error...',i,'J:',j)
        return(pd.DataFrame(
            data=
            [{'id':"",
              'title':"",
              'content':"",
              'excerpt':"",
              'createdAt':"",
              'updatedAt': "",
              'forumName':"",
              'forumAlias':"",
              'gender':"",
              'topics':""}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )

In [4]:
# KeyError: 'id'
# Oh！文章不見了 這篇文章已被作者刪除。 按這裡 看更多文章吧
# https://www.dcard.tw/f/makeup/p/235079399
# Catch(235079399)
# url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=233254532" 

In [6]:
import random
import time

#url = "http://dcard.tw/_api/forums/makeup/posts?popular=false&limit=1"
url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=229872382" 
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df = df.append(Catch(rejs[0]['id']),ignore_index=True) 
#df

In [ ]:
count = 0
delay_choices = [13,25,28,32]  # delay second
delay = random.choice(delay_choices) 

for j in range(300):
    last = str(int(df['id'].tail(1))) # 找出爬出資料的最後一筆ID
    print(last)
    url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=" + last
    resq = requests.get(url)
    rejs = resq.json()
    delay = random.choice(delay_choices)
    time.sleep(delay)  
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        count = count+1
        print('data reading...',count,'id:',rejs[i]['id'],rejs[i]['title'])
    df.to_csv(str(j) + '_Dcard.csv', index=False)
    
df.to_csv('data/Dcard_makeup.csv', index=False) #encoding = "UTF-8"
print(df.shape)
df


229849131
data reading... 1 id: 229848987 雙十 1028 👁👁
data reading... 2 id: 229848962 #問 A'pieu沙棘維生素C美白霜開啟時是否有封膜?
data reading... 3 id: 229848956 #圖 比粉刺大軍還要療癒的好物
data reading... 4 id: 229848864 在我臉上的大雷品-森田
data reading... 5 id: 229848850 第一次買專櫃
data reading... 6 id: 229848719 #圖 #分享 秋冬最美紅棕色🍂🍁MAC Marrakesh
data reading... 7 id: 229848707 ORBIS vs 植村秀
data reading... 8 id: 229848705 Colourpop 迪士尼公主眼影盤試色分享
data reading... 9 id: 229848554 #TKLAB折價券
data reading... 10 id: 229848547 #問 GA完美絲絨水慕斯粉底和蘭蔻新款的零粉感超持久粉底的色號比較
data reading... 11 id: 229848503 去除粉刺的產品
data reading... 12 id: 229848186 #請益 w.lab妝前保濕精華
data reading... 13 id: 229848113 #隱眼 #color scandal #09 blue
data reading... 14 id: 229848076 ＃問 #油肌 ＃很容易脫皮 （內有選手）
data reading... 15 id: 229848064 #分享 1028彩妝試色
data reading... 16 id: 229848047 #情報 媚比琳Fit me 反孔特霧無暇嫩粉餅
data reading... 17 id: 229847744 酒糟肌引起眼睛問題
data reading... 18 id: 229847520 蘭蔻零粉感色號請益
data reading... 19 id: 229847468 What’s in my makeup bag
data reading... 20 id: 229847393 請

data reading... 164 id: 229829968 #請益 開架蜜粉選擇 內有選手
data reading... 165 id: 229829933 💋👀購物狂🤦🏻‍♀️近期購入的大量唇彩和眼影盤😱
data reading... 166 id: 229829925 #分享 韓國氣墊控的各種美光～
data reading... 167 id: 229829885 好用的痘痘藥
data reading... 168 id: 229829882 #香水 雨天的香水-Miller Harris la pluie 驟雨詩篇
data reading... 169 id: 229829631 修容打亮特輯
data reading... 170 id: 229829599 #飄眉 這種技術高雄有嗎？
data reading... 171 id: 229829558 #請益 尋找自己色號的粉底液
data reading... 172 id: 229829336 #微更#分享  不愛用氣墊的我也被推坑 #maybelline 氣墊#容嬪
data reading... 173 id: 229829317 一盤眼影三個眼妝分享
data reading... 174 id: 229829271 Heme 唇膏 幫選色💄
data reading... 175 id: 229829238 #分享 Velcos 魅惑極光口紅
data reading... 176 id: 229829222 #黑特 知名美妝抽獎文 抽到心很冷😷
data reading... 177 id: 229829089 日本旅遊化妝包「小分享」！什麼都要帶小小的才有大大的空間放戰利品～～
data reading... 178 id: 229829064 上底妝後臉上的紋路？
data reading... 179 id: 229828913 雅漾 潔膚凝膠 洗臉的
data reading... 180 id: 229828881 Honey moon眼妝👀
data reading... 181 id: 229828814 I'm meme 我愛氣墊薄霧液態唇膏試色分享
data reading... 182 id: 229828738 #情報 植村秀潔顏油試用包五日份
data

data reading... 325 id: 229815038 有沒有推薦的化妝水（內有選手）
data reading... 326 id: 229815031 日常眼妝分享👀feat.蘭蔻 絕對完美迷霧唇膏
data reading... 327 id: 229814977 求推薦 Kiss隔離霜
data reading... 328 id: 229814841 Brick-o-la & bb cranberry 比較
data reading... 329 id: 229814818 #問#圖 可以代替青春露的好物
data reading... 330 id: 229814771 真的覺得有些粉絲太好騙
data reading... 331 id: 229814469 眉毛求批求指教
data reading... 332 id: 229814467 我的早八妝與保養步驟 #更新
data reading... 333 id: 229814415 #請益 週年慶粉底選擇
data reading... 334 id: 229814221 Laura mercier週年慶DM
data reading... 335 id: 229813926 #發問_液態唇彩選擇
data reading... 336 id: 229813892 #請益 專櫃底妝🙏🏻（內有蘭蔻、蘭芝、BB 選手)
data reading... 337 id: 229813809 請益 蝦皮李拉拉美妝
data reading... 338 id: 229813365 新手發問
data reading... 339 id: 229813338 #問 混合敏感肌隔離霜選擇
data reading... 340 id: 229813272 #黑特 周年慶滿額禮兌換前的陷阱
data reading... 341 id: 229813233 #情報 jo malone 聖誕組合
data reading... 342 id: 229813119 我的空瓶記
data reading... 343 id: 229812907 #分享 動物鼻貼🐯
data reading... 344 id: 229812737 請問SUGAO蜜粉
data reading... 345 id: 2298

data reading... 488 id: 229798261 #請益 卸妝蜜 sexylook新品 or za?
data reading... 489 id: 229798201 新手眉毛請益
data reading... 490 id: 229798159 屈臣氏加購美妝蛋
data reading... 491 id: 229798116 Tomod’s 戰利品
data reading... 492 id: 229798063 【指甲油】玫瑰色系指甲油分享
data reading... 493 id: 229797763 #小心得 暑假過了，犒賞一下自己吧
data reading... 494 id: 229797713 Catrice HD liquid coverage粉底液
data reading... 495 id: 229797708 新買口紅 凹進去
data reading... 496 id: 229797624 關於理膚duo+成分
data reading... 497 id: 229797621 💄分享// 我的保養清單♡
data reading... 498 id: 229797612 2018上半年防曬分享
data reading... 499 id: 229797574 眼影選擇 內有選手
data reading... 500 id: 229797537 #試色#更💄【Etude House X Red Velvet】水潤持久唇釉+偷渡愛用遮瑕
229797537
data reading... 501 id: 229797402 隱眼-挑選篇
data reading... 502 id: 229797360 #情報 週年慶買蘭蔻零粉感粉底組 首4日送10ml素顏霜～
data reading... 503 id: 229797355 拔隱眼
data reading... 504 id: 229797336 #問 只用清水洗臉 那卸妝怎麼辦
data reading... 505 id: 229797291 點數好用好處多👍
data reading... 506 id: 229797197 #問 make up for ever柔霧空氣粉餅
data reading... 507 id: 22979713

data reading... 648 id: 229782679 韓國隱眼 olens blossom ３con系列
data reading... 649 id: 229782022 近期的箱內遮瑕款
data reading... 650 id: 229782020 跟痘痘抗戰(有圖慎入)
data reading... 651 id: 229782003 The ordinary 杜鵑花酸
data reading... 652 id: 229781926 #圖多 #唇部試色 14支開架💄分享
data reading... 653 id: 229781824 #詢問 Anna sui 獨角獸香水
data reading... 654 id: 229781697 短額頭跟眉毛求解🙏🙏
data reading... 655 id: 229781540 名品會 線上outlet
data reading... 656 id: 229781484 #中原 #屈臣式 不知道是不是清貨❤️
data reading... 657 id: 229781300 終於湊滿Satin系列了！！！
data reading... 658 id: 229781209 Bobbi Brown🍒Cranberry🍒姐妹色
data reading... 659 id: 229781155 隱眼分享GEO BLENZ CHIC-BROWN 分享
data reading... 660 id: 229781123 女友的生日禮物
data reading... 661 id: 229781118 有人有急救辦法嗎😭
data reading... 662 id: 229781104 #更 Vxxxa 黑心代購 新限時聲明 重要連結微整理
data reading... 663 id: 229781100 真空除毛價格
data reading... 664 id: 229780629 請問有人買過這間嗎😊
data reading... 665 id: 229780622 第一次靠櫃買保養品
data reading... 666 id: 229780547 指甲貼
data reading... 667 id: 229780478 好用默默存在的糟糠妻💞碧兒泉黑膜力面膜
data 

...

data reading... 15704 id: 228023252 #問 暖橘眼盤抉擇
data reading... 15705 id: 228022609 #小禮物出現了！！
data reading... 15706 id: 228022530 #問 幫選擇colourpop 眼影（內有選手）
data reading... 15707 id: 228022254 請益！
data reading... 15708 id: 228022237 有人用23 years old 痘痘藥會脫皮￼嗎
data reading... 15709 id: 228022187 Ka'fen、控油洗髮精購買
data reading... 15710 id: 228022077 (問)眼線畫歪補救方法？
data reading... 15711 id: 228022063 #挑選 終極秋冬粉底2選1
data reading... 15712 id: 228021888 永和、中和微整形或是光療推薦淨膚雷射
data reading... 15713 id: 228021748 問colorpop 眼影盤推薦
data reading... 15714 id: 228021343 #請益 保濕又有點遮瑕度的氣墊粉餅選擇
data reading... 15715 id: 228021324 Pony effect八色眼影盤 x 跨年眼妝分享
data reading... 15716 id: 228021224 好像快過氣的Yes please盤（？單眼皮眼妝分享😂還有2隻唇彩
data reading... 15717 id: 228020417 #問 這款隱眼的名字
data reading... 15718 id: 228020368 請問有買meibe沒湊滿免運費送的東西是哪些呢？
data reading... 15719 id: 228020340 買氣墊送超可愛小樣😍//更✨
data reading... 15720 id: 228020219 #更 #圖 #開箱 韓國innisfreeXsnoopy lucky box 超可愛💁🏻
data reading... 15721 id: 228020166 #情報 Tiss卸妝油買一送一
data read

data reading... 15855 id: 228001679 Elf  超級平價歐美彩妝💕
data reading... 15856 id: 228001674 禦寒術～寒冷地區的好用小物們👍
data reading... 15857 id: 228001662 Chifure 唇膏開箱試色
data reading... 15858 id: 228001629 資生堂敏感防護4日組免費兌換！！！
data reading... 15859 id: 228001539 #情報 台中南區Canmake 03眼影盤
data reading... 15860 id: 228001366 #問 colourpop外包裝
data reading... 15861 id: 228001244 #實測10H#戀愛魔鏡牛奶美肌生粉餅#裸肌色
data reading... 15862 id: 228001186 #問#香水#請益 選擇困難😭
data reading... 15863 id: 228001126 蘭蔻黑天鵝防水款睫毛膏/Suqqu睫毛夾  分享
data reading... 15864 id: 228001065 天啊我一次收到15盤眼影!慘痛的colourpop購買經驗!
data reading... 15865 id: 228001052 #問 #求推薦清新的香水（內有選手）
data reading... 15866 id: 228000984 #問 唇膏💄有選手
data reading... 15867 id: 228000981 NXXXXX REPUBLIC 的新品?
data reading... 15868 id: 228000876 #情報 寶雅優惠
data reading... 15869 id: 228000695 #問 哥德式洗髮精
data reading... 15870 id: 228000668 推桃園遠百MAC天使櫃姐
data reading... 15871 id: 228000646 Essence 星光閃閃指甲油 分享
data reading... 15872 id: 228000485 #更 求救！使用霓淨思粉刺溶解精華後
data reading... 15873 id: 228000434 

data reading... 16010 id: 227978499 Nars特霧唇釉
data reading... 16011 id: 227978467 #眼妝
data reading... 16012 id: 227978358 秋冬最愛的三隻唇膏
data reading... 16013 id: 227978037 #問 搽保養品長痘痘？
data reading... 16014 id: 227977953 清潔✨✨✨
data reading... 16015 id: 227977838 詢問 肌研這瓶乳液
data reading... 16016 id: 227977728 #流團 香水分裝交換~~
data reading... 16017 id: 227977672 網狀雙眼皮貼問題
data reading... 16018 id: 227977649 #開箱 超用心120分的交換禮物🎁
data reading... 16019 id: 227977623 #問 maybelline 焦糖拿鐵
data reading... 16020 id: 227977556 #分享 買雅漾精華乳拿到的超佛組合！
data reading... 16021 id: 227977410 #更 Kathleen Lights Dream St.眼影盤試色分享💕
data reading... 16022 id: 227977235 #揪團 colourpop 湊免運 滿單不收
data reading... 16023 id: 227977044 #圖多 光療彩甲—10週年最棒的紀念💖
data reading... 16024 id: 227977027 GA 奢華訂製緞光水唇膏ecstasy shine #400
data reading... 16025 id: 227976961 #分享 Innisfree 聖誕指彩組+UNT壞女人
data reading... 16026 id: 227976795 #圖 推🙌 資生堂面皰洗面乳
data reading... 16027 id: 227976794 #情報 日本連鎖藥妝松本清要進駐台灣了
data reading... 16028 id: 227976744 擁有的粉底液色號
data 

data reading... 16160 id: 227959072 #情報 小藍罐買一送一💋
data reading... 16161 id: 227959019 打亮控❗Jeffree Star 24 KARAT 開箱
data reading... 16162 id: 227958919 #圖 解救雷底妝之M.A.C活力水噴霧
data reading... 16163 id: 227958895 #圖 美甲光療分享
data reading... 16164 id: 227958832 #金色閃亮疊擦！！LOREAL金色唇膏✨✨✨
data reading... 16165 id: 227958823 屈臣氏光療美甲貼1週使用心得
data reading... 16166 id: 227958787 細軟髮的一片天！
data reading... 16167 id: 227958588 分享 剪了一顆陳小希的頭（圖多）
data reading... 16168 id: 227958512 #情報 Loreal 純色訂製唇膏玫瑰風暴系列
data reading... 16169 id: 227958397 #情報 契爾氏打卡抽禮物🎁
data reading... 16170 id: 227958341 #禮物 首次參加就換到天使🎁
data reading... 16171 id: 227958139 #innisfree #叮叮噹聖誕限定指彩組
data reading... 16172 id: 227958115 #情報 康是美折價券
data reading... 16173 id: 227958058 睫毛夾選手
data reading... 16174 id: 227957954 選Zoeva眼影盤 （內有選手）
data reading... 16175 id: 227957753 Wet n Wild Silk Finish Lipstick  534B
data reading... 16176 id: 227957483 #圖 🍂8支秋冬南瓜/土色不專業手部試色
data reading... 16177 id: 227957355 青春淨痘露可以當收斂水嗎
data reading... 16178 id: 22795722

data reading... 16313 id: 227935147 #問 角鯊&堅果油滋潤程度
data reading... 16314 id: 227935058 #圖 Make up for ever藝術大師性感尤霧唇釉301（近似Nars slow ride)
data reading... 16315 id: 227934944 #揪團 #新年團 開放式/專櫃彩妝保養交換
data reading... 16316 id: 227934910 #慎入#圖多 我爹的陳年粉刺林
data reading... 16317 id: 227934900 打亮選擇（內有選手
data reading... 16318 id: 227934871 2017 五支愛用唇彩💄Maybelline/Chifure/L'oreal/MAC
data reading... 16319 id: 227934793 #圖 聖誕節的時尚鼻毛妝
data reading... 16320 id: 227934733 #揪團  #關表單 淘寶受受狼刷具😱
data reading... 16321 id: 227934699 男香控香水收藏分享4：誘惑撲倒篇-2
data reading... 16322 id: 227934559 蘭芝唇膜顏色不一樣(更)
data reading... 16323 id: 227934463 庫洛魔法使刷具 來源
data reading... 16324 id: 227934423 #反推 綠藤生機「非乳液」話術
data reading... 16325 id: 227934404 #問 #蘭芝塑眉盤
data reading... 16326 id: 227934375 ＃情報 現在Dr.ci:labo官方Line上可以換小橘水28ml
data reading... 16327 id: 227934334 維密唇彩 #分享 #圖💄
data reading... 16328 id: 227934301 推薦北上霧眉地方
data reading... 16329 id: 227934276 #圖 #試色 #stila
data reading... 16330 id: 227934229 尋找DUO+味的香水
data reading..

In [ ]:
import random
import time

count = 0
delay_choices = [3,7,5]  # delay second
delay = random.choice(delay_choices) 

for j in range(5):
    print('data reading...',j,delay)
    delay = random.choice(delay_choices)
    time.sleep(delay)  

# ---------------正式版本分隔線---------------
- 測試版本代號：1.2  2020
- 正式版本代號：2.03  2021
- 開發人員： EGG Jerry Three


dva.x_x 版本開發

In [ ]:
#找到 dav.x_x 發表過的文章

p = requests.Session()                             # 把這一次的request都算在同一個session裡
url=requests.get("https://www.dcard.tw/@dva.x_x")  # 請求的網址
soup = BeautifulSoup(url.text,"html.parser")
sel = soup.select("h2.tgn9uw-2 a")
sel
a=[]
for link in sel:
    a.append(link["href"])
    print(link.get('href')[-9:])
    print('https://www.dcard.tw'+link.get('href'))

In [ ]:
# slope the title
def CatchUser(num):
    try:
        url = 'https://www.dcard.tw/_api/posts/'+str(num)  #api post with ID
        resq = requests.get(url)
        rejs = resq.json()
        return(pd.DataFrame(
            data=
            [{'id':rejs['id'],
              'department':rejs['department'],  #for author
              'title':rejs['title'],
              'content':rejs['content'],
              'excerpt':rejs['excerpt'],
              'createdAt':rejs['createdAt'],
              'updatedAt': rejs['updatedAt' ],
              'forumName':rejs['forumName'],
              'forumAlias':rejs['forumAlias'],
              'gender':rejs['gender'],
              'topics':rejs['topics']}],
            columns=['id', 'department', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )
    except:
        print('data error...',i,'J:',j)
        return(pd.DataFrame(
            data=
            [{'id':"",
              'department':"",    #for author
              'title':"",
              'content':"",
              'excerpt':"",
              'createdAt':"",
              'updatedAt': "",
              'forumName':"",
              'forumAlias':"",
              'gender':"",
              'topics':""}],
            columns=['id', 'department', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )

In [ ]:
# 熱門美妝版測試 (100筆)
url = 'https://www.dcard.tw/@dva.x_x'
#url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=5'
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
for i in range(len(rejs)):
    df = df.append(CatchUser(rejs[i]['id']),ignore_index=True)   #0 - 29
print(df.shape)
#df.to_csv('data/Op_Dcard_markeup.csv', index=False)
df
# 99	235031315	#請益 #問 LA girl 遮瑕挑色
# 235066991

In [ ]:
import random
import time

url = "http://dcard.tw/_api/forums/makeup/posts?popular=false&limit=1"
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df = df.append(CatchUser(rejs[0]['id']),ignore_index=True) 
#df
print(str(int(df['id'].tail(1))) )

count = 0
delay_choices = [1, 3, 5]  # delay second
delay = random.choice(delay_choices) 

for j in range(1):
    last = str(int(df['id'].tail(1))) # 找出爬出資料的最後一筆ID
    print(last)
    url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=" + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        count = count+1
        print('data reading...',count,'id:',rejs[i]['id'],rejs[i]['title'])
    delay = random.choice(delay_choices)
    time.sleep(delay)  
df.to_csv('data/Op_Dcard_markeup.csv', index=False)
print(df.shape)
df

In [ ]:
# slope the title
def CatchPost(num):
    try:
        url = 'https://www.dcard.tw/_api/posts/'+str(num)  #api post with ID
        resq = requests.get(url)
        rejs = resq.json()
        return(pd.DataFrame(
            data=
            [{'id':rejs['id'],
              'title':rejs['title'],
              'content':rejs['content'],
              'excerpt':rejs['excerpt'],
              'createdAt':rejs['createdAt'],
              'updatedAt': rejs['updatedAt' ],
              'forumName':rejs['forumName'],
              'forumAlias':rejs['forumAlias'],
              'gender':rejs['gender'],
              'topics':rejs['topics']}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )
    except:
        print('data error...',i,'J:',j)
        return(pd.DataFrame(
            data=
            [{'id':"",
              'title':"",
              'content':"",
              'excerpt':"",
              'createdAt':"",
              'updatedAt': "",
              'forumName':"",
              'forumAlias':"",
              'gender':"",
              'topics':""}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )

In [ ]:
#CatchPost(234999065)
#url = 'https://www.dcard.tw/_api/posts/'+str(num)  #api post with ID
url = 'https://www.dcard.tw/_api/posts/235103419'
resq = requests.get(url)
rejs = resq.json()
rejs

In [ ]:
url = 'https://www.dcard.tw/_api/posts/234912821'
resq = requests.get(url)
rejs = resq.json()
rejs

In [ ]:
# url = 'https://www.dcard.tw/_api/posts/235103302'
resq = requests.get(url)
rejs = resq.json()
rejs

# --------------- dva.x_x 版本分隔線 ---------------
- 測試版本代號：Jerry 1.0
- 開發人員： Jerry


In [ ]:
#test = open("data/test.txt","w",encoding='UTF-8')

In [ ]:
# asb
import random
import time

url = "http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=99"
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df = df.append(Catch(rejs[0]['id']),ignore_index=True)   #0 - 29

delay_choices = [1]#, 3, 5, 7,20]  #延遲的秒數
delay = random.choice(delay_choices)  #隨機選取秒數

for j in range(10):
    last = str(int(df['id'].tail(1))) # 找出爬出資料的最後一筆ID
    url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=true&limit=99&before=" + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        if i % 101 == 0:
            print('第',j,'次執行，data reading...',i,'last',last,'chocice',delay)
    delay = random.choice(delay_choices)  #隨機選取秒數
    time.sleep(delay)  #延遲

df.to_csv('data/Op_Dcard_markeup.csv', index=False)
print(df.shape)
df

In [ ]:
# 熱門美妝版測試 (100筆)
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100'
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
for i in range(len(rejs)):
    df = df.append(Catch(rejs[i]['id']),ignore_index=True)   #0 - 29
    if i % 20 == 0:
        print('data reading...',i)
print(df.shape)
df.to_csv('data/Op_Dcard_markeup.csv', index=False)
df
# 99	235031315	#請益 #問 LA girl 遮瑕挑色
# 235066991

In [ ]:
#last = str(int(df['ID'].tail(1))) #last dt
#last

In [ ]:
# 透過迴圈讀取10*100篇文章，若需讀取更多資料，可以將range(10)中的數值提升
for j in range(10):
    last = str(int(df['ID'].tail(1))) # 找出爬出資料的最後一筆ID
    url = 'https://www.dcard.tw/_api/posts?popular=true&before=' + last
    #url = 'https://www.dcard.tw/_api/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        if i % 10 == 0:
            print('第',j,'次執行，data reading...',i)
            
print(df.shape)
df

In [ ]:
last = str(int(df['ID'].tail(1)))
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df

In [ ]:
# 熱門美妝版測試 (100筆)

for i in range(10):
    last = str(int(df['ID'].tail(1)))
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    df = pd.DataFrame()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']),ignore_index=True)
        if i % 20 == 0:
            print('data reading...',i)
    print(df.shape)

df.to_csv('data/Op_Dcard_markeup.csv', index=False)
df

# 99	235031315	#請益 #問 LA girl 遮瑕挑色

In [ ]:
last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID '235031315'
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
resq = requests.get(url)
rejs = resq.json()
for i in range(len(rejs)):
    df = df.append(Catch(rejs[i]['id']), ignore_index=True)
print(df.shape)


df

In [ ]:

#df

In [ ]:
##ok ^
for j in range(3):
    last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
print(df.shape)
#df

In [ ]:
# 透過迴圈讀取10*100篇文章，若需讀取更多資料，可以將range(10)中的數值提升
for j in range(10):
    last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        if i % 20 == 0:
            print('data reading...',i)
print(df.shape)
df


In [ ]:
df.to_excel('data/Dcard_markeup.excel', index=False)

欄位	說明	備註
ID	編號	
title	標題	
content	內容	
excerpt	摘要	
createdAt	發布時間	
updatedAt	更新時間	
commentCount	留言數	
forumName	分類	中文
forumAlias	分類	英文
gender	性別	
likeCount	心情數量	
reactions	心情細項	把以上心情細分為「愛心」、「哈哈」、「跪」、「森77」、「驚訝」等類型
topics	標籤	

In [ ]:
# 固定碼測試
def Catch_test(ID):
    url = 'https://www.dcard.tw/_api/posts/'+str(ID)
    requ = requests.get(url)
    rejs = requ.json()
    #rejs
    return(pd.DataFrame(
        data=
        [{'ID':rejs['id'],
          'title':rejs['title'],
          'content':rejs['content'],
          'excerpt':rejs['excerpt'],
          'createdAt':rejs['createdAt'],
          'updatedAt': rejs['updatedAt' ],
          'forumName':rejs['forumName'],
          'forumAlias':rejs['forumAlias'],
          'gender':rejs['gender'],
          'topics':rejs['topics']}],
        columns=['ID', 'title', 'content', 'excerpt', 'createdAt', 
                 'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
          )
Catch_test(235020702)